## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/data/c_elegan_pair1.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings
fix_spacing = np.array([0.4, 0.1625, 0.1625])
mov_spacing = np.array([0.4, 0.1625, 0.1625])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Masking
---

### fixed

In [2]:
# tools for coarse whole brain segmentation
from CircuitSeeker import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# fix_skip = fix[::4, ::8, ::8]
# skip_spacing = fix_spacing * [4, 8, 8]
# fix_mask_small = level_set.brain_detection(
#     fix_skip, skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,2],
#     lambda2=1.0,
# )

# # enlarge and smooth mask
# fix_mask = zoom(fix_mask_small, np.array(fix.shape) / fix_skip.shape, order=0)
# fix_mask = binary_closing(fix_mask, np.ones((5,5,5))).astype(np.uint8)
# fix_mask = binary_dilation(fix_mask, np.ones((5,5,5))).astype(np.uint8)

# # write result
# nrrd.write('./fix_mask.nrrd', fix_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
fix_mask, _ = nrrd.read('./fix_mask.nrrd')
fix_mask = fix_mask.transpose(2,1,0)

### moving

In [3]:
# tools for coarse whole brain segmentation
from CircuitSeeker import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# mov_skip = mov[::4, ::8, ::8]
# skip_spacing = mov_spacing * [4, 8, 8]
# mov_mask_small = level_set.brain_detection(
#     mov_skip, skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,2],
#     lambda2=4.0,  # 1.0 a little too small
# )

# # enlarge and smooth mask
# mov_mask = zoom(mov_mask_small, np.array(mov.shape) / mov_skip.shape, order=0)
# mov_mask = binary_closing(mov_mask, np.ones((5,5,5))).astype(np.uint8)
# mov_mask = binary_dilation(mov_mask, np.ones((5,5,5))).astype(np.uint8)

# # save output
# nrrd.write('./mov_mask.nrrd', mov_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
mov_mask, _ = nrrd.read('./mov_mask.nrrd')
mov_mask = mov_mask.transpose(2,1,0)

## Alignment
---

### global

In [4]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':1.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(1.5,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
}

affine_kwargs = {
    'initial_condition':'CENTER',
}

affine2_kwargs = {
    'fix_mask':fix_mask,
    'mov_mask':mov_mask,
    'optimizer_args':{
        'learningRate':1.0,
        'minStep':0.,
        'numberOfIterations':3600,
    },
}

# TODO: deform not needed for this sample
deform_kwargs = {
    'fix_mask':fix_mask,
    'mov_mask':mov_mask,
    'control_point_spacing':32.0,
    'control_point_levels':(1,),
    'alignment_spacing':1.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(1.5,),
    'optimizer_args':{
        'learningRate':10.0,
        'minStep':0.,
        'numberOfIterations':60,
    },
}

steps = [('affine', {**common_kwargs, **affine_kwargs},),
         ('affine', {**common_kwargs, **affine2_kwargs},),
         ('deform', {**common_kwargs, **deform_kwargs},),]

# align
affine, deform = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
    return_format='compressed',
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# deform aligned
deform_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine, deform],
)

# # write results
np.savetxt('affine.mat', affine)
nrrd.write('./deform.nrrd', deform, compression_level=2)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)
nrrd.write('./deformed.nrrd', deform_aligned.transpose(2,1,0), compression_level=2)

# # load precomputed results
# affine = np.loadtxt('./affine.mat')
# # deform, _ = nrrd.read('./deform.nrrd')

LEVEL:  0  ITERATION:  0  METRIC:  -0.2012940925622078
LEVEL:  0  ITERATION:  1  METRIC:  -0.20350897568787019
LEVEL:  0  ITERATION:  2  METRIC:  -0.2047004608475962
LEVEL:  0  ITERATION:  3  METRIC:  -0.2045955432014801
LEVEL:  0  ITERATION:  4  METRIC:  -0.20474139934922087
LEVEL:  0  ITERATION:  5  METRIC:  -0.2058087411220701
LEVEL:  0  ITERATION:  6  METRIC:  -0.20694243970517787
LEVEL:  0  ITERATION:  7  METRIC:  -0.20662800373314846
LEVEL:  0  ITERATION:  8  METRIC:  -0.20747443092204657
LEVEL:  0  ITERATION:  9  METRIC:  -0.2081030302051964
LEVEL:  0  ITERATION:  10  METRIC:  -0.20810917265531656
LEVEL:  0  ITERATION:  11  METRIC:  -0.20920296804651772
LEVEL:  0  ITERATION:  12  METRIC:  -0.20947264457911471
LEVEL:  0  ITERATION:  13  METRIC:  -0.20963167870592775
LEVEL:  0  ITERATION:  14  METRIC:  -0.21041743206368815
LEVEL:  0  ITERATION:  15  METRIC:  -0.21087988830605287
LEVEL:  0  ITERATION:  16  METRIC:  -0.21131965924228918
LEVEL:  0  ITERATION:  17  METRIC:  -0.2119058